## Load Libraries

In [1]:
import json
from random import randint, sample
import pandas as pd
import numpy as np
import pickle
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.utils import shuffle
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import MinMaxScaler

import xgboost
import treelite


sns.set(style="ticks")
np.set_printoptions(suppress=True)

pd.set_option('display.max_rows', 150)

## Load Data

In [2]:
with open('/data-temp/data/nonwear-check/O/results/annotations.json', 'r') as f:
    annotations = json.load(f)

record_annotation_index = {}
for record_annotation in annotations['record_annotations']:
    if record_annotation['id'] not in record_annotation_index:
        record_annotation_index[record_annotation['id']] = {}
    id = record_annotation['id']
    record_annotation.pop('id')
    record_annotation_index[id].update(record_annotation)

segment_annotation_index = {}
for segment_annotation in annotations['segment_annotations']:
    if segment_annotation['id'] not in segment_annotation_index:
        segment_annotation_index[segment_annotation['id']] = {}
    id = segment_annotation['id']
    segment_annotation.pop('id')
    segment_annotation_index[id].update(segment_annotation)

In [3]:
df_features = pd.read_csv("/data-temp/data/nonwear-check/O/results/features__ppg-g__object_length_36__cut_500.csv", index_col=None)
df_features = df_features.iloc[shuffle(range(len(df_features)), random_state=0), :]

df_objects = pd.read_csv("/data-temp/data/nonwear-check/O/results/objects__ppg-g__object_length_36__cut_500.csv", index_col=None)

In [4]:
feat_cols = [c for c in df_features.columns if "ppg" in c]
target_col  = "wear_category_id"

## 全数据构建模型

In [52]:
# 利用xgboost选择前50个特征分析
params = {'max_depth': 3, 'objective':'binary:logistic'}
num_iter = 5

X_cols, y_col = ['ppg__autocorrelation__lag_1', 'ppg__agg_linear_trend__attr_"intercept"__chunk_len_10__f_agg_"mean"'], target_col

X, y = df_features.loc[:, X_cols].values, df_features.loc[:, y_col].values

# scaler = MinMaxScaler()
# X = scaler.fit_transform(X)

D_train = xgboost.DMatrix(X, label=y)
D_test = xgboost.DMatrix(X, label=y)

bst = xgboost.train(params, D_train, num_iter, [(D_train, 'train')], verbose_eval=False)

accuracy_score(y, bst.predict(D_test) > 0.7)

0.9840496233938857

In [53]:
model = treelite.Model.from_xgboost(bst)

model.export_srcpkg(platform='unix', toolchain='gcc', pkgpath='./mymodel.zip',
                    libname='mymodel.so', verbose=True)

[14:55:47] /io/treelite/src/frontend/xgboost.cc:359: Global bias of the model: 0.5
[14:55:47] /io/treelite/src/frontend/xgboost.cc:397: gbm_param_.num_feature = 0
[14:55:47] /io/treelite/src/frontend/xgboost.cc:398: gbm_param_.num_output_group = 0
[14:55:47] /io/treelite/src/compiler/ast_native.cc:22: Using ASTNativeCompiler
[14:55:47] /io/treelite/src/compiler/ast/split.cc:10: Parallel compilation disabled; all member trees will be dumped to a single source file. This may increase compilation time and memory usage.
[14:55:47] /io/treelite/src/c_api/c_api.cc:297: Code generation finished. Writing code to files...
[14:55:47] /io/treelite/src/c_api/c_api.cc:314: Writing file recipe.json...
[14:55:47] /io/treelite/src/c_api/c_api.cc:314: Writing file main.c...
[14:55:47] /io/treelite/src/c_api/c_api.cc:314: Writing file header.h...


## 验证和C一致性

In [56]:
selected_object_ids = list(range(3000, 3004))

### 待验证数据

In [57]:
mask = np.isin(df_objects['id'], selected_object_ids)
mask = np.where(mask)[0]
df_objects.loc[mask, 'ppg'].values

array([-10704.,   8296.,  25344.,  22336.,   7984.,  11760.,  31592.,
        56560.,  38152.,  16032.,  -3736., -15560.,  -5024.,  11384.,
        27976.,  25224.,   9552.,   9760.,  26888.,  59856.,  40656.,
        13760.,  -5528., -10416.,  -8096.,   5400.,  22408.,  26104.,
        12704.,   5664.,  15096.,  47472.,  52752.,   8616.,    240.,
        -7880.,  -2416.,   7048.,  17784.,  22384.,  12800.,   7040.,
        20160.,  54536.,  49720.,   2904.,  -7576.,  -6168.,  -3080.,
         5280.,  19464.,  22368.,  12024.,   8048.,  29176.,  56640.,
        37696.,   3384.,  -7432., -15200.,   -976.,  17608.,  25264.,
        13840.,   6336.,  22440.,  54904.,  43920.,   8360.,   1232.,
       -10072.,  -5664.,   7528.,  19664.,  23592.,  15656.,   5472.,
         9032.,  42168.,  39936.,  26288.,   3024.,  -9408.,  -1152.,
        11760.,  22608.,  25176.,  11976.,   2456.,  24552.,  56632.,
        24976.,  -4720.,  -5400.,  -6248.,   6800.,  17776.,  30144.,
        18760.,   54

### 特征一致性

In [58]:
features = df_features.loc[selected_object_ids, X_cols]
features

,ppg__autocorrelation__lag_1,"ppg__agg_linear_trend__attr_""intercept""__chunk_len_10__f_agg_""mean"""
3000,0.561048,-4982.482520
3001,0.569601,-4983.175513
3002,0.497899,-4980.859497
3003,0.590755,-4970.489111


### 模型一致性

In [59]:
D = xgboost.DMatrix(features.values)
bst.predict(D)

array([0.87677187, 0.87677187, 0.8768642 , 0.87677187], dtype=float32)

In [66]:
import json
import logging

_logger = logging.getLogger('nni')

In [67]:
_logger.info(
            "Creating graph json, writing to. Visualization enabled.")